## Computational Theory Assessment

In [47]:
import numpy as np

## Problem 1: Binary Words and Operations

**Brief:** Implement the following functions in Python. Use numpy to ensure that all variables and values are treated as 32-bit integers. These functions are defined in the Secure Hash Standard (see page 10). Document each function with a clear docstring, explain its purpose and behaviour in Markdown, and test it with appropriate examples to verify correctness.

### Problem 1 Introduction:

The Secure Hash Standard (FIPS 180-4) defines a family of cryptographic hash algorithms, including **SHA-224, SHA-256, SHA-384,** and **SHA-512**. These algorithms use carefully designed bitwise operations to achieve diffusion and non-linearity, which are critical for cryptographic security.
This section of the Problems.ipynb document provides detailed documentation for the core bitwise operations used in the SHA-256 cryptographic hash algorithm, as specified in FIPS PUB 180-4 (Secure Hash Standard). The functions in this section (Problem 1) implement the operations which form the fundamental building blocks of SHA-256's compression function and message schedule expansion, as defined in the Secure Hash Standard.

**SHA-256 Core Functions - Comprehensive Documentation** 

SHA-256 achieves cryptographic security through:

* Diffusion: Small input changes cause large output changes
* Non-linearity: Complex relationships between inputs and outputs
* Avalanche effect: One bit change affects approximately 50% of output bits

The functions in this section implement the following primitives as defined in **Section 4 of the Secure Hash Standard**:

* u_32: Type conversion which ensures correct 32-bit arithmetic
* Ch, Maj, Parity: Logical functions which provide non-linear mixing
* Sigma functions: Rotation and shift operations which provide diffusion

#### **u_32(x)**
**Purpose:**
Enforces 32-bit unsigned integer arithmetic required by the SHA-256 specification - essentially this function ensures all arithmetic and bitwise operations stay within 32-bit unsigned integer bounds. All SHA-256 operations must use modulo 2^32 arithmetic to ensure consistent, reproducible hash values across all platforms and implementations- essentailly, all SHA operations must use unsigned 32-bit integers to produce correct hash values

**Why This Is Essential:** Python’s integers are unbounded by default, which would produce incorrect SHA-256 hashes. SHA-256 uses modulo 2^32 arithmetic for every operation which forces values into a 32-bit type. 

This function ensures:

* Correct overflow behavior: Values wrap around at 2^32−1 (4,294,967,295)
* Proper masking: All values stay within 32-bit bounds
* Platform consistency: Same results on all systems
* Specification compliance: Matches FIPS 180-4 requirements exactly

**Usage in SHA-256**

This function is the foundation of all SHA-256 operations:

* Applied to inputs before any bitwise operation
* Applied to intermediate results during computation
* Applied to final outputs to ensure correct format
* Must be used consistently throughout the algorithm

Below, the u_32(x) function implements this logic by using the NumPy library's uint32 type to convert a value to an unsigned 32-bit integer. It’s used here to ensure that integer values are:
* exactly 32 bits wide
* non-negative

As described in NumPy's official documentation, uint32 stores exactly 32 bits, meaning it can represent values from 0 to 2^32−1 (4,294,967,295). NumPy ensures that a value x is in the representable range (a 32-bit unsigned integer) by applying modulo 2^32 arithmetic, which means forcing a number to fit into 32 bits by keeping only its lowest 32 bits if the value goes out of range (i.e. once x goes past 2^32 − 1). Essentially, for any integer x, applying modulo 2^32 means computing: x mod 2^32, which gives the remainder after dividing x by 2^32. The result is always in the range: 0 ≤ x mod 2^32 < 2^32

Specific cases:
* If x is a negative value: NumPy applies modulo 2^32 wrapping. −1 wraps to 2^32 − 1 (4,294,967,295)
* If x is larger than 2^32 − 1, NumPy discards higher-order bits. 2^32 + 10 wraps to 10


In [48]:
def u_32(x):
    """
    Purpose: convert a value to an unsigned 32-bit integer using NumPy's uint32 data type

    This function acts as a helper function to ensure that all SHA operations use
    unsigned 32-bit integers in order to produce correct hash values, as stated in
    the requirements of SHA-256's 32-bit modular arithmetic.
    This function must be applied consistently to prevent overflow and ensure 
    correct hash computation. 
    
    How the function works: The conversion wraps values (x) by applying modulo 2^32 and 
    retaining only the lowest 32 bits. Negative values and values larger than 2^32 - 1
    are automatically wrapped.

    Parameters
    ----------
    x : int
        Input integer value to be converted to an unsigned 32-bit integer
        
    Returns
    -------
    np.uint32
        Value of x when converted to an unsigned 32-bit integer with modulo 2^32 wrapping
    """

    # returns the value of x converted to an unsigned 32-bit integer using uint32 
    return np.uint32(x)

#### **Parity(x, y, z)**

**Purpose** 

Parity is a simple nonlinear combination used in some hashing and cryptographic constructions (although SHA-256 does not use Parity, it appears in SHA-1). It serves as a symmetric bitwise mixing function by computing x ⊕ y ⊕ z using the XOR bitwise operator, meaning each output bit is 1 if an odd number of inputs have that bit set. Essentially, the Parity function determines whether an odd or even number of bits are set at each bit position across three 32-bit inputs.

For each bit position:

* Output is 1 if an odd number (1 or 3) of inputs have 1
* Output is 0 if an even number (0 or 2) of inputs have 1

**Mathematical Definition:** Parity(x, y, z) = x ⊕ y ⊕ z

**Usage Context**

This Parity operation is used within the SHA-1 algorithm to introduce mixing and complexity during the hash computation process in SHA-1 rounds 20-39 and 60-79. Parity is not used SHA-256. It is included here for:

* Completeness: Understanding the SHA family
* Comparison: Contrast with SHA-256's Ch and Maj functions
* Educational value: Demonstrates simpler non-linear mixing

SHA-256 replaced Parity with the more complex **Ch** and **Maj** functions (defined later in this section) for stronger security.

**Implementation**

The Parity operation is implemented as a function Parity(x, y, z) in this section (Problem 1), and uses the bitwise XOR operation, as it is defined in the official python documentation, to determine whether an odd or even number of bits are set at each bit position across three 32-bit inputs (x, y, z). Parity(x, y, z) applies the XOR operation twice as follows:

* Step 1: x ⊕ y → Produces 1 where x and y differ
* Step 2: (x ⊕ y) ⊕ z → Produces 1 where the result differs from z

Each bit of the final result tells you the parity of the corresponding bits of x, y, and z. It creates a parity check: the final bit is 1 if an odd number of input bits are 1.

**Relationship to Other Functions**

* Simpler than Ch: Treats all inputs symmetrically (no selector)
* Simpler than Maj: No majority voting, just XOR
* Pure XOR: Fully reversible and linear
* Weaker security: This is why SHA-256 uses Ch and Maj instead

In [ ]:
def Parity(x, y, z):
    """
    Purpose: compute the Parity function used in SHA-1.
    
    Implements the XOR parity operation as defined in FIPS 180-4 for SHA-1.
    This function performs bitwise XOR on three inputs to determine parity:
    returns 1 when an odd number of corresponding bits are set, 0 otherwise.
    
    The operation proceeds as follows:
    1. XOR compares corresponding bits of x and y:
       - Output 1 if bits differ (one 1, one 0)
       - Output 0 if bits match (both 0 or both 1)
    This result represents partial parity for x and y:
       - Bit is 0 → even number of 1s (0 or 2)
       - Bit is 1 → odd number of 1s (1)
    
    2. XOR then compares this result with z:
       - Output 1 if bits differ
       - Output 0 if bits match
    
    3. Final result shows parity across all three inputs:
       - Bit is 1 → odd number of 1s (1 or 3)
       - Bit is 0 → even number of 1s (0 or 2)

    Parameters
    ----------
    x, y, z : int
        32-bit integer values
        
    Returns
    -------
    np.uint32 
        Result of x ⊕ y ⊕ z
    
    """
    # returns the unsigned 32-bit integer result of the XOR of x, y, and z, which are converted to unsigned 32-bit integers using u_32() helper function
    return u_32(x) ^ u_32(y) ^ u_32(z)

#### **Ch(x, y, z)** — Choose Function

**Definition (FIPS 180-4):** Ch(x, y, z) = (x ∧ y) ⊕ (¬x ∧ z)

**Purpose**
Ch implements SHA-256's conditional selection function where x acts as a selector to "choose" bits from either y or z. This creates complex, non-linear relationships essential for cryptographic security.

**Implementation**

The Ch(x, y, z) function implements the Choose (Ch) function as defined in the Secure Hash Standard FIPS 180-4. It computes the conditional selection of a 32-bit value. The Ch() function chooses bits from either of the two input integers y and z, based on the value of input x. Essentially, this function uses x as a selector to choose bits from either y or z: 

* if a bit in x is 1, then the corresponding bit from y is chosen
* if a bit in x is 0, then the corresponding bit from x is chosen.

In the implementation of the Ch(x, y, z) function, this is done by using the AND bitwise operation on x and y, then using the NOT operation on x, before applying AND to x and z.

* First, AND compares each bit in both values being considered and only outputs 1 when both bits are 1. If only one bit is 1 or neither bits are 1, a 0 is output. Therefore where x has 1s, the corresponding bits from y are output, and where x has 0s, 0s are output. 
* Next, When performing the AND operation on x and z, the NOT operation is first used to flip all the bits in x (1→0, 0→1). When AND is performed on (NOT)x and z, the bits where x orignally had 0s (now 1s) outputs the corresponding bits of z, and the bits where x originally had 1s (now 0s) outputs 0s. 
* Finally, the XOR bitwise operation is performed on the resulting values of x AND y, and (NOT)x and z. XOR is used to compare two values by outputing 1 when bits are different. Therefore when XOR is applied, we get a binary number which shows which bits are different the results of both AND operations performed on x y and x z. However, due to the face that NOT was applied when calculating the result of x and z, but not when calculating the result of x and y, this results in the outputs of these operations being mutually exlusive - there is no overalp between the two: each result has 1s where the other has 0s and vice versa. So, in this situation, XOR simply acts like OR and combines the two results. XOR is used because...

As stated in the Secure Hash Standard, this produces an ouput which is very difficult to reverse-engineer as it depends on all three inputs in a complex way: small changes in x can cause the function to switch between y and z. This creates the avalanche effect crucial for cryptographic security.

**Usage in SHA-256** 

Used in the main compression function, rounds 0-63, where its applied to working variable e in each round

Purpose in algorithm: T1 = h + Σ₁(e) + Ch(e, f, g) + K[t] + W[t]

Where e, f, g are working variables: Ch selects between f and g based on e, which creates complex dependencies between state variables

**Cryptographic Properties**

* Non-linearity: Output depends on all three inputs in complex ways
* Avalanche effect: Small changes in x can completely change the final output (switches between y and z)
* Asymmetry: x has special role as selector (unlike Maj where all inputs are equal)
* Differential resistance: Prevents attackers from predicting how bit changes propagate

**Relationship to Other Functions**

* Different from Maj: Ch treats inputs asymmetrically (x is selector)
* Different from Parity: Ch is more complex, provides stronger mixing
* Complements Maj: SHA-256 uses both for different purposes
* Works with Σ₁: Applied together in compression function

In [ ]:
def Ch(x, y, z):
    
    """
    Purpose: compute the Choose (Ch) function used in SHA-256.
    
    Implements the conditional selection operation from FIPS 180-4 Section 4.1.2.
    The Ch function uses x as a bitwise selector to choose bits from either y or z,
    creating non-linear mixing essential for SHA-256's security.
    
    Operation Mechanics
    -------------------
    For each of the 32 bit positions:
    - If bit in x is 1 → select corresponding bit from y
    - If bit in x is 0 → select corresponding bit from z
    
    Operation Proceedings
    --------------------
    1. Compute (x ∧ y): Extracts bits from y where x = 1
       - AND compares each bit in x and y
       - Outputs 1 only when both are 1
       - Result: y's bits where x = 1, zeros elsewhere
    
    2. Compute ¬x: Flip all bits in x (1→0, 0→1)
    
    3. Compute (¬x ∧ z): Extracts bits from z where x = 0
       - AND compares each bit in ¬x and z
       - Result: z's bits where x was 0, zeros elsewhere
    
    4. Combine with XOR: (x ∧ y) ⊕ (¬x ∧ z)
       - The two parts are mutually exclusive (no overlap)
       - XOR acts like OR, combining both selections
       - Result: Perfect bit-by-bit selection

    Parameters
    ----------
    x, y, z : int  
      32-bit integer values
        
    Returns
    -------
    np.uint32 
      Result of (x ∧ y) ⊕ (¬x ∧ z) converted into an unisgned 32-bit integer 
    
    """

    # the result of x AND y is combined (using XOR) with the result of (NOT) x AND z
    # the result is converted to an unsigned 32-bit integer value and returned
    return u_32((u_32(x) & u_32(y)) ^ (~u_32(x) & u_32(z)))

#### **Maj(x, y, z)** — Majority Function

**Purpose** 

Maj (the majority function), as defined in the Secure Hash Standard, is a bitwise Boolean function which makes up part of the SHA-256 compression function, where it is used in each round to update the hash state. It is used to mix bits in a non-linear and balanced way by outputting the majority bit of three input words at each bit position which imrpoves resistance to cryptanalysis. This is because the majority function is balanced and treats all inputs equally, but changing one input only changes the output where that input is the minority, and the relationship between inputs and outputs is not a simple and linear combination which makes it harder to decrypt. 

**Mathematical Definition:** Maj(x, y, z) = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)

**Implementation**

The Maj(x, y, z) function implements SHA-256's majority voting function Maj where each output bit represents the majority value (0 or 1) among the three input bits at that position:

* If 2 or more inputs have 1 → output is 1
* If 2 or more inputs have 0 → output is 0

Essentially what it does is compare each of the corresponding bits in x, y, and z and if 2 or more of the inputs have a 1, the output is 1. If 2 or more of the inputs have a 0, the output is 0. In other words: the bit value that appears most frequently is output - hence majority vote. To achieve this majority vote, the Maj(x, y, z) function (implemented below) follows these steps:

* Step 1: the AND bitwise operation is performed on (x and y), (x and z), and (y and z) respectfully. For each pair, each corresponding bit in the two values are compared: if the corresponding bits are the same, 1 is output. If they are different, 0 is output. This is essentially where the two values "vote together" for 1.
* Step 2: the first two results of these operations are combined using XOR: (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z), meaning each corresponding bit in the three results are compared and if the bits are the same, then 0 is ouput. If one or more bits are different, then the output is 1. 

Essentially XOR outputs 1 when there's an odd number of 1s in its inputs:

* Majority is 1 (2+ ones) → Odd pairs agree (1 or 3) → XOR outputs 1 
* Majority is 0 (2+ zeros) → Even pairs agree (0) → XOR outputs 0 

**Usage in SHA-256**

Maj is used in the main compression function, all 64 rounds and is applied to working variable a in each round

Purpose in algorithm: T2 = Σ₀(a) + Maj(a, b, c)

Where: a, b, c are working variables that Maj combines through majority voting, where the result updates the state in next round

* Ch: Applied to variables (e, f, g)
* Maj: Applied to variables (a, b, c)

**Cryptographic Properties**

* Symmetric: All three inputs treated equally (no special roles)
* Balanced: Output evenly distributed (50% ones, 50% zeros)
* Non-linear: Relationship between inputs and outputs is complex
* Smooth transitions: Changing one input affects output only where it's in minority
* Democratic: Consensus-based, no single input dominates

**Relationship to Other Functions**

* Different from Ch: Maj treats all inputs equally, meaning it is symmetric, while Ch is asymetric
* Different from Parity: Maj does majority voting, not just XOR as XOR alone is linear and vulnerable to attacks

SHA-256 uses both Ch and Maj for complete mixing because they provide different types of non-linearity: 
* Ch: Asymmetric, selector-based (x controls output)
* Maj: Symmetric, voting-based (all equal)


Works with Σ₀: as it will be discussed later in this section, Maj and Sigma0 are applied together in the compression function as using both ensures complete mixing of all working variables so there are no patterns or symmetries attackers can exploit, therefore rendering the result resistant to various cryptanalytic techniques

In [ ]:
def Maj(x, y, z):
    """
    Purpose: compute the Majority (Maj) function used in SHA-256.
    
    Maj operates on three 32-bit value inputs (x, y, z) and outputs a single 32-bit value,
    which is the the result of a combination of bitwise AND and XOR operations used to determine “majority” 
    value among the three inputs.
    - If 2 or more inputs have 1 → output 1 (majority votes 1)
    - If 2 or more inputs have 0 → output 0 (majority votes 0)
    - The bit value appearing most frequently wins
    
    Parameters
    ----------
    x, y, z : int 
        32-bit integer values
        
    Returns
    -------
    np.uint32
        Result of (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)
        
    """

    # returns the result of the XOR operation which combines the results of the AND operation performed on the unsigned 32-bit integers:
    # (x and y), (x and z), and (y and z)
    return u_32((u_32(x) & u_32(y)) ^ (u_32(x) & u_32(z)) ^ (u_32(y) & u_32(z)))

#### **rotr(x, n)** — Right Rotation

**Purpose:** perform a right rotation on a 32-bit value. rort(x, n) rotates bits instead of shifting in zeros which isn a core operation in SHA-256’s mixing steps. rotr(x, n) is different from a logical shift because instead of bits being dropped once they go out of range, the rotation wraps around the dropped bits and adds them back onto the beginning of the vale. This preserves all bit information while performing a diffusion to improve encryption, which is the main function of rotr(x, n). Rotations are a core building block of the **Σ** and **σ** functions (implemented later in this section), so this helper function rotr(x, n) (shown below) implements rotations to facilitate these functions.

**Why rotations matter:**
* They are nonlinear with respect to shifts
* They ensure every output bit depends on multiple positions of the input
* They preserve all bits (unlike shifts, which lose information)

**Implementation**

The function rotr(x, n) implements the rotation as follows: **(x≫n)∣(x≪(32−n))**

The rotr(x, n) implements the operation ROTR^n(x), as defined in the Secure Hash Standard, which is a circular shift where all bits are shifted to the right and added back onto the begging when they go out of range. This is how the function achieves this:

* The bits which are shifted out of range off the right end of the integer, using right shift (>>), wrap around to the left and are added back onto the beginning of the integer using by combining the right shift (>>) with a left shift (<<) operation using the OR bitwise operation "|". 
* This allows for all bits which are shifted off the right end to be added back onto the left end of the 32-bit value. This preserves all bit information while performing a diffusion to improve encryption

In [ ]:
def rotr(x, n):
    """
    Purpose: perform a right rotation on a 32-bit value.
    
    This function implements ROTR^n(x) to perform a circular shift where all bits are shifted to 
    the right and added back onto the begging when they go out of range by combining
    the right shift (>>) with a left shift (<<) operation using the OR bitwise operation "|". 

    Parameters
    ----------
    x : int 
        32-bit integer value to rotate
    n : int 
        Number of positions to rotate right (0 ≤ n ≤ 32)
        
    Returns
    -------
    u_32((x >> n) | (x << u_32(32 - n))): 
        The rotated value of x by n bits converted to an unsigned 32-bit integer

    """

    # convert x to an unsigned 32-bit integer using u_32() helper funciton
    x = u_32(x)

    # return the 32-bit integer value of the circular right rotation of x by n bits
    return u_32((x >> n) | (x << u_32(32 - n)))

### **Sigma Functions**

The Secure Hash Standard defines four Sigma/sigma function which are used in the SHA-256 compression function to introduce strong diffusion and ensure that small changes in input values result in widespread changes in the internal state of the hash function which is essential for cryptographic security. Essentially, by combining multiple rotated versions of the same word, the functions ensure that each output bit depends on multiple input bits. The four sigma functions operate on 32-bit words using combinations of bitwise rotations (ROTR) and logical shifts (SHR) as specified in NIST FIPS 180-4.
The sigma functions are divided into two categories:

* **Uppercase Σ (Σ₀, Σ₁):** Used in the compression function to mix the working variables.
* **Lowercase σ (σ₀, σ₁):** Used in the message schedule expansion to spread message bits across rounds.

All sigma functions operate on unsigned 32-bit integers and must be evaluated using modulo 2^32 arithmetic, as required by the Secure Hash Standard.

#### **Sigma0(x)** — Uppercase **Σ₀**

Definition (FIPS 180-4): **(x)=ROTR^2(x) ⊕ ROTR^13(x) ⊕ ROTR^22(x)**

**Purpose:** Σ₀ is a function used in the SHA-256 compression function to non-linearly mix the working variable a. It is applied repeatedly inside the compression function.Its purpose is to provide strong diffusion during each round of compression, ensuring that changes to the internal state propagate quickly and unpredictably.

**Implementation** 

The Sigma0(x) function implements the Sigma0 operation as defined in the Secure Hash Standard FIPS 180-4. Sigma0 combines three different rotations in order to achieve strong bit diffusion. It does this by applying three different rotations to the input variable x using ROTR^n(), where n represents the number of rotations performed on the input value x, and then combines the results of these rotations using XOR. The Sigma0 function below implements this functionality with the following steps:

* Step 1: the first rotation ROTR^2, rotates the bits in x by 2: meaning each bit in x is shifted to the right by 2, and the bits that go out of range on the right side of the x value are wrapped around and added back onto the left side of the x value, therefore ensuring a circular right rotation where no bits are lost.
* Step 2: The second rotation used on x is ROTR^13, which shifts the bits in x by 13 and ensures that the values that go out of range are wrapped onto the beggining of x to preserve all bits. 
* Step 3: The third rotation used on x is ROTR^22, which shifts the bits in x by 22 and ensures that the values that go out of range are wrapped onto the beggining of x to preserve all bits. 
* Step 4: The results of these three rotations are combined using XOR. This compares all corresponding bits in all values and outpus 1 if there is an odd number of 1s present (1 or 3) at this bit across the three rotations and outputs 0 if there are an even number of 1s. 

As stated in the Secure Hash Standard, the implementation of Sigma0 combines three different rotations to provide strong bit diffusion in the working variable updates. The specific rotation amounts (2, 13, 22) were chosen to maximize diffusion and resist cryptanalysis, as the rotation amounts sum to 37, which is prime, ensuring good bit distribution across multiple rounds. 

**Usage in SHA-256**

The Secure Hash Standard specifies Σ₀ as part of the computation of the temporary value: **T2​=Σ0​(a)+Maj(a,b,c)**

**Cryptographic Properties**

**Relationship to Other Functions**


In [ ]:
def Sigma0(x):
    """
    Purose: compute the Sigma0 (Σ₀) function for SHA-256 compression
    
    This function applies three different rotations to the input variable x using the helper 
    function rotr(x, n). Sigma0 takes the input, rotates it right by three different amounts 
    (2, 13, and 22), then applies XOR tp combine all three rotated versions.
       
    Parameters
    ----------
    x : int
        32-bit integer value
        
    Returns
    -------
    np.uint32: 
        Result of ROTR^2(x) ⊕ ROTR^13(x) ⊕ ROTR^22(x)

    """
    
    # convert the input value x to an unsigned 32-bit value to ensure it is handled correctly 
    x = u_32(x)
    
    # returns the result of performing 3 rotations on input x using helper method rotr()
    # a rotation of 2, 13, and then 15 are applied to x using rotr()
    # and then using XOR the three results are compared to produce the final 32-bit value result
    return rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22)

#### **Sigma1(x)** — Uppercase **Σ₁**
Definition (FIPS 180-4): **(x)=ROTR^6(x) ⊕ ROTR^11(x) ⊕ ROTR^25(x)**

**Purpose:**

Σ₁ is a function used in the SHA-256 compression function to non-linearly mix the working variable e in each round of SHA-256's main compression loop. Like Σ₀, it is applied repeatedly inside the compression function and it is intended to work complementarily with Σ₀ to provide enhanced bit diffusion during each round of compression, ensuring that changes to the internal state propagate quickly and unpredictably.

**Implementation**

This function implementing Sigma1 follows the same logic as the function implementing Sigma0: three different rotations are applied to the input variable x using the helper function rotr(x, n), where n represents the number of rotations performed on the input value x, and then the results of these rotations are combined using XOR. Sigma1 implements this functionality as follows:

* Step 1: The first rotation ROTR^6 is applied to x - it rotates the bits in x by 6
* Step 2: The second rotation used on x is ROTR^11, which shifts the bits in x by 11
* Step 3: The third rotation used on x is ROTR^25, which shifts the bits in x by 25
* Step 4: The results of these three rotations are combined using XOR. This compares all corresponding bits in all values and outpus 1 if there is an odd number of 1s present (1 or 3) at this bit across the three rotations and outputs 0 if there are an even number of 1s. 

The rotation amounts (6, 11, 25) differ from Sigma0 to ensure different diffusion patterns. These amounts 6, 11, and 25, were specifically chosen by NIST through extensive cryptanalysis to resist known attack vectors.

**Usage in SHA-256** 

It provides diffusion and non-linearity during the update of the internal state and is part of the computation of: T1​=h+Σ1​(e)+Ch(e,f,g)+Kt​+Wt​

This ensures that the evolution of the state depends on multiple rotated views of e.


In [ ]:
def Sigma1(x):
    """
    Purpose: compute the Sigma1 (Σ₁) function for SHA-256 compression
    
    This function applies three different rotations to the input variable x using the helper 
    function rotr(x, n). Sigma1 takes the input, rotates it right by three different amounts 
    (6, 11, and 25), then applies XOR tp combine all three rotated versions.
    
    Parameters
    ----------
    x : int 
        32-bit integer value
        
    Returns
    -------
    np.uint32: 
        Result of ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)
        
    """

    # convert the input value x to an unsigned 32-bit value to ensure it is handled correctly 
    x = u_32(x)

    # return the result of performing 3 rotations of 6, 11, and 25, on input x using helper method rotr() and then extracting a final value with XOR
    return u_32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))

#### **sigma0(x)** — Lowercase **σ₀**
Small mixing function using rotations and shifts: **(x)=ROTR^7(x) ⊕ ROTR^18(x) ⊕ (x≫3)**

Used when expanding the message schedule array.

**Purpose:** Mix input words to create additional message schedule values & introduce nonlinearity early, before compression

**Note:** uses a logical right shift in addition to rotations.

Works in conjunction with sigma1 in the message schedule equation: W[t] = σ₁(W[t-2]) + W[t-7] + σ₀(W[t-15]) + W[t-16]

In [55]:
def sigma0(x):
    """
    Function's purpose: compute the sigma0 (σ₀) function for SHA-256 message schedule.
    
    This function implements the sigma0 function as defined in the Secure Hash Standard
    FIPS 180-4, which is used to generate words 16-63 of the message schedule from the
    original 16 words of the message block. sigma0 applies two different rotations to 
    the input variable x using ROTR^n(x), where n represents the number of rotations 
    performed on the input value x, and then uses the right shift bitwise operation 
    (n >> x) to shift the bits of x to the right, where n represents the number of bit 
    shifts performed on x. Then, the three results of these operations are combined
    using XOR.
        - The first rotation performed on x is ROTR^7 which rotates the bits in x by 7: 
        meaning each bit in x is shifted to the right by 7, and the bits that go out of 
        range on the right side of the x value are wrapped around and added back onto the 
        left side of the x value, therefore ensuring a circular right rotation where no 
        bits are lost.
        - The second rotation used on x is ROTR^18, which shifts the bits in x by 18
        and ensures that the values that go out of range are wrapped onto the beggining
        of x to preserve all bits. 
        - The third operation performed on x is a right shift of 3, which sifts all bits
        in x to the right by an amount of 3, causing the bits that go out of range to be
        lost and three 0s are added on to the start of x to pad the value and preserve its
        32-bit state. 
        - The final result is achieved by using XOR to combine the result of each of the 
        three operations performed on x. This operation produces an output by comparing
        the corresponding bits in all three values and outpus 1 if there is an odd number 
        of 1s present (1 or 3) at this bit across the three rotations and outputs 0 if 
        there are an even number of 1s. 
    sigma0 is set apart from both uppercase Sigma operations by a critical distinction:
    The final operation is right SHIFT (>>), not rotation. This means the top 3 bits become 0, 
    which introduces asymmetry that enhances the avalanche effect.
    
    Parameters
    ----------
    int x : 32-bit integer value
        
    Returns
    -------
    np.uint32: Result of ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)
    
    """

    # convert the input value x to an unsigned 32-bit value to ensure it is handled correctly 
    x = u_32(x)
    
    # return the result of performing the following operations on input x:
    # 2 rotations of 7, and then 18, using helper method rotr() and a right shift of 3 
    # the final 32-bit value result is found by using XOR to combine the three results from the operations
    return u_32(rotr(x, 7) ^ rotr(x, 18) ^ (x >> 3))

#### **sigma1(x)** — Lowercase **σ₁**
Small mixing function using rotations and shifts: **(x)=ROTR^17(x) ⊕ ROTR^19(x) ⊕ (x≫10)**

**Purpose:** Also part of the message schedule expansion. Helps propagate entropy from earlier message words into later ones.

In [56]:
def sigma1(x):
    """
    Function's purpose: compute the sigma1 (σ₁) function for SHA-256 message schedule

    This function implements the sigma1 function as defined in the Secure Hash Standard
    FIPS 180-4, which complements sigma0 in generating the extended 64-word message schedule.
    sigma1 applies two different rotations to the input variable x using ROTR^n(x), where 
    n represents the number of rotations performed on the input value x, and then uses 
    the right shift bitwise operation (n >> x) to shift the bits of x to the right, where 
    n represents the number of bit shifts performed on x. Then, the three results of these 
    operations are combined using XOR.
        - The first rotation performed on x is ROTR^17 which rotates the bits in x by 17: 
        meaning each bit in x is shifted to the right by 17, and the bits that go out of 
        range on the right side of the x value are wrapped around and added back onto the 
        left side of the x value, therefore ensuring a circular right rotation where no 
        bits are lost.
        - The second rotation used on x is ROTR^19, which shifts the bits in x by 19
        and ensures that the values that go out of range are wrapped onto the beggining
        of x to preserve all bits. 
        - The third operation performed on x is a right shift of 10, which sifts all bits
        in x to the right by an amount of 10, causing the bits that go out of range to be
        lost and ten 0s are added on to the start/left side of x to pad the value and preserve 
        its 32-bit state. 
        - The final result is achieved by using XOR to combine the result of each of the 
        three operations performed on x. This operation produces an output by comparing
        the corresponding bits in all three values and outpus 1 if there is an odd number 
        of 1s present (1 or 3) at this bit across the three rotations and outputs 0 if 
        there are an even number of 1s. 
    The combination of rotations and shifts ensures that each bit of the original message 
    influences many bits in the final hash.
    Like sigma0, sigma1 is set apart from both uppercase Sigma operations by a critical distinction:
    The final operation is right SHIFT (>>), not rotation. This means the top 10 bits become 0, 
    which introduces asymmetry that enhances the avalanche effect, therefore contributing to the 
    non-linear properties of the message schedule.
    
    Parameters
    ----------
    int x : 32-bit integer value
        
    Returns
    -------
    np.uint32: Result of ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)
    
    """
    # convert the input value x to an unsigned 32-bit value to ensure it is handled correctly 
    x = u_32(x)
    # return the result of performing the following operations on input x:
    # 2 rotations of 17, and then 19, using helper method rotr() and a right shift of 10
    # the final 32-bit value result is found by using XOR to combine the three results from the operations
    return u_32(rotr(x, 17) ^ rotr(x, 19) ^ (x >> 10))

### **How these functions work together**

Ch, Maj, Sigma0, and Sigma1, combine in each of the 64 rounds:

Simplified SHA-256 round: 

* T1 = h + Σ₁(e) + Ch(e, f, g) + K[t] + W[t]
* T2 = Σ₀(a) + Maj(a, b, c)

Update working variables

h = g

g = f

f = e

e = d + T1

d = c

c = b

b = a

a = T1 + T2

### **Problem 1 Tests**

#### ***Testing u_32() helper function***

In [57]:
print("=== Testing unsigned_32 ===")
value = u_32(np.uint32(0xffffffff) + np.uint32(1))
print("Input: 0xFFFFFFFF + 1")
print("32-bit wrapped output:", u_32(value))
print("Expected: 0x00000000\n")

=== Testing unsigned_32 ===
Input: 0xFFFFFFFF + 1
32-bit wrapped output: 0
Expected: 0x00000000



/tmp/ipykernel_2909/2094878837.py:2: RuntimeWarning: overflow encountered in scalar add
  value = u_32(np.uint32(0xffffffff) + np.uint32(1))


#### ***Testing Parity() function***

In [58]:
print("\n=== Testing Parity ===")
print("Input: 0b1010, 0b0101, 0b1100")
print("Output:", bin(Parity(np.uint32(0b1010), np.uint32(0b0101), np.uint32(0b1100))))
print("Expected: 0b1010 ^ 0b0101 ^ 0b1100 =", bin(0b1010 ^ 0b0101 ^ 0b1100))


=== Testing Parity ===
Input: 0b1010, 0b0101, 0b1100
Output: 0b11
Expected: 0b1010 ^ 0b0101 ^ 0b1100 = 0b11


#### ***Testing Ch() function***

In [59]:
print("\n=== Testing Ch ===")
print("If x bit = 1 → choose y bit")
print("If x bit = 0 → choose z bit\n")
print("Example:")
print("x = 0xFFFFFFFF, y = 0x12345678, z = 0x87654321")
print("Output:", hex(Ch(np.uint32(0xffffffff), np.uint32(0x12345678), np.uint32(0x87654321))))
print("Expected: y → 0x12345678")


=== Testing Ch ===
If x bit = 1 → choose y bit
If x bit = 0 → choose z bit

Example:
x = 0xFFFFFFFF, y = 0x12345678, z = 0x87654321
Output: 0x12345678
Expected: y → 0x12345678


#### ***Testing Maj() function***

In [60]:
print("\n=== Testing Maj ===")
print("Example bits: x=1, y=1, z=0 → majority is 1")
print("Example bits: x=0, y=0, z=1 → majority is 0\n")
print("Input: 0b1010, 0b1100, 0b1000")
print("Output:", bin(Maj(np.uint32(0b1010), np.uint32(0b1100), np.uint32(0b1000))))
print("Expected majority:", bin(0b1000))


=== Testing Maj ===
Example bits: x=1, y=1, z=0 → majority is 1
Example bits: x=0, y=0, z=1 → majority is 0

Input: 0b1010, 0b1100, 0b1000
Output: 0b1000
Expected majority: 0b1000


#### ***Testing rotr() helper function***

In [61]:
print("\n=== Testing rotr ===")
print("Input: x = 0b0001, n = 1")
print("Output:", bin(rotr(np.uint32(0b0001), 1)))
print("Expected: 0x80000000 bit pattern")
print("\nInput: x = 0x12345678, n = 4")
print("Output:", hex(rotr(np.uint32(0x12345678), 4)))
print("Expected:", hex(0x81234567))


=== Testing rotr ===
Input: x = 0b0001, n = 1
Output: 0b10000000000000000000000000000000
Expected: 0x80000000 bit pattern

Input: x = 0x12345678, n = 4
Output: 0x81234567
Expected: 0x81234567


#### ***Testing Sigma0() function***

In [62]:
print("\n=== Testing Sigma0 ===")
x = np.uint32(0x6a09e667)
print("Input: x =", hex(x))
print("Output:", hex(Sigma0(x)))
print("This mixes the word using 3 rotations.")


=== Testing Sigma0 ===
Input: x = 0x6a09e667
Output: 0xce20b47e
This mixes the word using 3 rotations.


#### ***Testing Sigma1() function***

In [63]:
print("\n=== Testing Sigma1 ===")
x = np.uint32(0xbb67ae85)
print("Input: x =", hex(x))
print("Output:", hex(Sigma1(x)))
print("Used every round inside SHA-256 compression.")


=== Testing Sigma1 ===
Input: x = 0xbb67ae85
Output: 0x758db092
Used every round inside SHA-256 compression.


#### ***Testing sigma0() function***

In [ ]:
print("\n=== Testing sigma0 ===")
x = u_32(0x428a2f98)
print("Input:", hex(x))
print("Output:", hex(sigma0(x)))
print("This is used in the message schedule.")


=== Testing sigma0 ===
Input: 0x428a2f98
Output: 0xb332410e
This is used in the message schedule (W[t]).


#### ***Testing sigma1() function***

In [65]:
print("\n=== Testing sigma1 ===")
x = np.uint32(0x71374491)
print("Input:", hex(x))
print("Output:", hex(sigma1(x)))
print("Also used in message schedule expansion.")


=== Testing sigma1 ===
Input: 0x71374491
Output: 0x4ac6db6c
Also used in message schedule expansion.


## Problem 2: Fractional Parts of Cube Roots
**Brief:** Use numpy to calculate the constants listed at the bottom of page 11 of the Secure Hash Standard, following the steps below. These are the first 32 bits of the fractional parts of the cube roots of the first 64 prime numbers.
1. Write a function called primes(n) that generates the first n prime numbers.
2. Use the function to calculate the cube root of the first 64 primes.
3. For each cube root, extract the first thirty-two bits of the fractional part.
4. Display the result in hexadecimal.
5. Test the results against what is in the Secure Hash Standard.



## Problem 2 Introduction:

We've seen how the functions from Problem 1 work in the SHA-256 compression function. What other aspects are there to the compression function? This is the subject of Problem 2: Implementing ... as defined in the Secure Hash Standard
**The Secure Hash Standard (FIPS PUB 180-4)** defines **SHA-256** as part of the **SHA-2** family of cryptographic hash functions. **SHA-256** operates on 32-bit words and relies on:
* Initial hash values (H0–H7)
* Round constants (K0–K63)

Both are derived from irrational numbers to ensure unpredictability and uniform bit distribution.
**Specifically:** Initial hash values **(H0–H7)** are the first 32 bits of the fractional part of the square roots of the first 8 prime numbers. Round constants **(K0–K63)** are the first 32 bits of the fractional part of the cube roots of the first 64 prime numbers.

The code below generates the **SHA-256** round constants (K) step by step.

#### **primes(n)** Function:

* Generates the first n prime numbers.
* Starts from 2 and checks divisibility to determine primality.

Used to obtain deterministic, well-known numbers as input to the cube-root and square-root operations in **SHA-256**. Why in SHA-256: Prime numbers are the basis for generating constants to reduce predictability and avoid hidden patterns.

In [66]:
def primes(n):
    primes = []
    num = 2
    while len(primes) < n:
        for i in range(2,num):
            if num % i == 0:
                break
        
            primes.append(num)
        num += 1
    return primes

#### **cube_roots(primes)** Function:
Computes the cube root of each prime. **SHA-256** specifies that round constants **(K0–K63)** are based on the fractional part of the cube roots of the first 64 primes.

In [67]:
def cube_root(primes):
    cube_roots = [] 
    cube_roots = np.cbrt(primes)
    return cube_roots

#### **frac_32()** Function:

* Extracts the fractional part of the cube roots using np.modf().
* Scales the fractional part to 32 bits (frac * 2^32) and converts to 32-bit unsigned integers (np.uint32).
* Produces the K constants used in each SHA-256 round.

**Why in SHA-256:** Using the fractional part ensures a uniform, non-repeating bit pattern for cryptographic mixing.

In [68]:
def frac_32():
    p = primes(64)
    cube_roots = cube_root(p)
    frac, _= np.modf(cube_roots)
    constants = np.floor(frac * (2**32)).astype(np.uint32)
    return constants 

#### **hex_conversion(constants)** Function:
Converts the 32-bit integers into 8-character hexadecimal strings, zero-padded. **SHA-256** specifications and reference implementations often display constants in hexadecimal. Makes constants readable and comparable against the official standard values.

In [69]:
def hex_conversion(constants):
    hex_constants = []
    for c in constants:
        hex_constants.append(hex(c)[2:].zfill(8))
    return hex_constants

In [70]:
K = np.array([
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174, 
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da, 
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967, 
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
], dtype=np.uint32)

In [71]:
## test primes
prime_nums = primes(64)
print(prime_nums)
print(len(prime_nums))

## test cubes
cubes = cube_root(prime_nums)
print(cubes)

## test fracs 
frac, nonfrac= np.modf(cubes)
print("fractional:", frac)
print("non fractional:", nonfrac)

## test constants
constants = np.floor(frac * (2**32)).astype(np.uint32)
print("const:", constants)

## test hex constants
hex_constants = hex_conversion(constants)
print("Hex constants", hex_constants)

def compare_constants(official_constants, hex_constants):
    return np.array_equal(official_constants, hex_constants)

print(len(K))
print(len(K))
print(K)
print(compare_constants(K, hex_constants))

[3, 5, 5, 5, 7, 7, 7, 7, 7, 9, 11, 11, 11, 11, 11, 11, 11, 11, 11, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 15, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 21]
64
[1.44224957 1.70997595 1.70997595 1.70997595 1.91293118 1.91293118
 1.91293118 1.91293118 1.91293118 2.08008382 2.22398009 2.22398009
 2.22398009 2.22398009 2.22398009 2.22398009 2.22398009 2.22398009
 2.22398009 2.35133469 2.35133469 2.35133469 2.35133469 2.35133469
 2.35133469 2.35133469 2.35133469 2.35133469 2.35133469 2.35133469
 2.46621207 2.57128159 2.57128159 2.57128159 2.57128159 2.57128159
 2.57128159 2.57128159 2.57128159 2.57128159 2.57128159 2.57128159
 2.57128159 2.57128159 2.57128159 2.57128159 2.66840165 2.66840165
 2.66840165 2.66840165 2.66840165 2.66840165 2.66840165 2.66840165
 2.66840165 2.66840165 2.66840165 2.66840165 2.66840165 2.66840165
 2.66840165 2.66840165 2.66840165 2.75892418]
fractional: [0.44224957 0.70997595

## Problem 3: Padding

**Brief:** Write a generator function block_parse(msg) that processes messages according to section 5.1.1 and 5.2.1 of the Secure Hash Standard. The function should accept a bytes object called msg. At each iteration, it should yield the next 512-bit block of msg as a bytes object. Ensure that the final block (or final two blocks) include(s) the required padding of msg as specified in the standard. Test the generator with messages of different lengths to confirm proper padding and block output.

###  Problem 3 Introduction

Before hashing can begin, the message needs to be prepared in a specific way - everything needs to be the right size and format. This preparation has three steps, and we're focusing on the first two: padding and parsing.

#### Padding the Message
The goal: make the message length a specific multiple (either 512 bits or 1024 bits, depending on the SHA algorithm used).

**For SHA-1, SHA-224, and SHA-256 (512-bit blocks):**

If the message is ℓ bits long here's what happens:
* Add a "1" bit to the end of your message
* Add k zero bits, where k is chosen so that (ℓ + 1 + k) equals 448 when you divide by 512 and look at the remainder
* Add a 64-bit representation of ℓ (the original message length)

**The result: (ℓ + 1 + k + 64) is always 512 bits.**

#### Parsing the Message

Once padded, you need to break the message into chunks (blocks) for processing.
For **SHA-256** and similar algorithms, you split your padded message into N blocks of 512 bits each. Each 512-bit block is then subdivided into sixteen 32-bit words (since 16 × 32 = 512).

So if you have block i, it contains words M₀⁽ⁱ⁾, M₁⁽ⁱ⁾, M₂⁽ⁱ⁾, ... M₁₅⁽ⁱ⁾.

In [72]:
def file_to_msg(filepath):
    # Reading a file in binary mode
    f = open(filepath, 'rb')
    msg = f.read()
    return msg

In [73]:
def block_parse(msg):
    no_bytes=64
    # Initialize bit counter
    no_bits = len(msg) * 8
    position = 0

    # split msg in 512-bit (64 byte) blocks & yield them
    while position < len(msg):
        block = msg[position:position+no_bytes]
        # check that the block is not a partial block - stops last block from being yielded 
        if len(block) == no_bytes:
            position += no_bytes
            yield block
        else:
            break

    # breaking out of the while loop when a partial block is found means block variable does not contain that partial block
    # get the remaining bytes in the partial block:
    block = msg[position:]  # This will be empty if position == len(msg)
    
    # check that the msg bytes objec is not empty - if it is, assign empty bytes object to last (and only) block
    if no_bits == 0:
        block = bytes()

    # Scenario 1: are there at least 9 bytes available?
    if (64 - len(block)) >= 9:
       yield (block
              + bytes([0x80])
              + bytes([0x00] * (64 - len(block) - 1 - 8))
              + no_bits.to_bytes(8, byteorder='big'))
       
    # Scenario 2: there are between 8 and 1 bytes available, inclusive.
    elif (64 - len(block)) >= 1:
       yield block + bytes([0x80]) + bytes([0x00] * (64 - len(block) - 1))
       yield bytes([0x00] * 56) + no_bits.to_bytes(8, byteorder='big')
       
    # Scenario 3: There were no bytes available in the last block.
    else:
       yield bytes([0x80] + ([0x00] * 55)) + no_bits.to_bytes(8, byteorder='big')

## Problem 4: Hashes

**Brief:** Write a function hash(current, block) that calculates the next hash value given the current hash value and the next message block according to section 6.2.2 SHA-256 Hash Computation on page 22 of the Secure Hash Standard.

### Introduction Problem 4:

In [74]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

def hash(current, block):
    # Read block as a array of 32-bit unsigned ints in big endian.
    block = np.frombuffer(block, dtype='>u4') 

    # Assign room for a 64 element 32-bit int array.
    W = np.zeros(64, dtype=np.uint32)

    # The first 16 elements of W come from the message block.
    for t in range(16):
        W[t] = block[t]
    #
    for t in range(16, 64):
        W[t] = sigma1(W[t-2]) + W[t-7] + sigma0(W[t-15]) + W[t-16]
        
    # assign current hash values to 8 variables a->h
    a = current[0]
    b = current[1]
    c = current[2]
    d = current[3]
    e = current[4]
    f = current[5]
    g = current[6]
    h = current[7]

    # loop where: T1 & T2 calculated & values of h->a get cascaded down to next next variable
    # this is done to ensure original data (message) is well-mixed with an asymetric hash 
    for t in range(64):
        T1 = h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t]
        T2 = Sigma0(a) + Maj(a, b, c)
        h = g
        g = f
        f = e
        e = d + T1
        d = c
        c = b
        b = a
        a = T1 + T2

    # calculate new hash using hashed variables a->h
    H = np.array([
        a + current[0], b + current[1], c + current[2], d + current[3],
        e + current[4], f + current[5], g + current[6], h + current[7],
    ], dtype=np.uint32)

    return H

# get first 8 hash codes from official hash codes of the secure hash standard (Section 5.3.3)
H = np.array([
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19,
], dtype=np.uint32)

# get message from file
msg = file_to_msg('abc.txt')

# get padded blocks from block_parse() & loop through them
for block in block_parse(msg):
    # for each block, apply the hash
    H = hash(H, block) 

# test
msg = b"abc"
for block in block_parse(msg):
    H = hash(H, block) 

# print the result in hex (more readable format)
result = ''.join(f'{x:08x}' for x in H)
print("Result: ", result)
print("Expected:   ", "ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad")
print("Match:", result == "ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad")


Result:  1de51655e1a8ca5f570c062c6fef497dd8110762bd8a7a48dd95482f7d22e0a2
Expected:    ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
Match: False


## Problem 5: Passwords